# Variational Autoencoder (VAE): "non-linear PCA"

## VAE Resources I'm using

- `https://www.tensorflow.org/tutorials/generative/cvae`
- `https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE`
- `https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/vae.py`

## VAE Resources I'm not using here, but found easily
- `https://keras.io/examples/generative/vae/`
- `https://www.tensorflow.org/guide/keras/custom_layers_and_models#putting_it_all_together_an_end-to-end_example`

# Some Standard Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets
import matplotlib.pyplot as plt


# MNIST digits
- I initially started with a tutorial using the `keras` style here, i.e., not TFDS
    - but we're going to end up convertig this to TFDS, so we could have just started with that

In [2]:
# https://www.tensorflow.org/tutorials/generative/cvae
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE
if 0:
    import tensorflow_datasets as tfds
    datasets, datasets_info = tfds.load(name='mnist', with_info=True,
                                        as_supervised=False) # this would give us (x,x) as opposed to (x,y)

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
train_images.shape, test_images.shape

In [ ]:
i=0
plt.imshow(train_images[i,], cmap=plt.cm.binary)

In [ ]:
train_images[0,:10,:10]

## TF Dataset

- This original tutorial I was looking at did it's transform of the data all up front

In [ ]:
# https://www.tensorflow.org/tutorials/generative/cvae

if 0:
    def preprocess_images(images):
        images = images.reshape((images.shape[0], 28, 28, 1)) / 255.
        return np.where(images > .5, 1.0, 0.0).astype('float32')

    train_images_bernoulli = preprocess_images(train_images)
    test_images_bernoulli = preprocess_images(test_images)

- We'll instead incorporate this into the TFDS iterator and process it on the fly

In [ ]:
 # turn into a tf Dataset

dataset = {}
dataset['train'] = tf.data.Dataset.from_tensor_slices(train_images)
dataset['test'] = tf.data.Dataset.from_tensor_slices(test_images)

next(iter(dataset['train'])).numpy()[:10,:10]

- **We'll set it up as (x,x), not (x,y)**
- Binarizing lets make the image a grid of bernoulli outcomes
    - By preprocessing they'll vary each time so we get **data augmentation**

In [ ]:
# https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE

batch_size = 32

def _preprocess(sample):
                           #tf.float32 is important!
    image = tf.cast(sample, tf.float32) / 255.  # Scale to unit interval. 
    
    # https://www.tensorflow.org/api_docs/python/tf/reshape
    image = tf.reshape(image, [28, 28, 1])
    
    # Randomly binarize [Each Time Each Batch!]
    image = image > tf.random.uniform(tf.shape(image))   
    return image, image    # (x,y) is actually (x,x)!

train_dataset = (dataset['train']
                 .map(_preprocess)
                 .batch(batch_size)
                 .prefetch(tf.data.experimental.AUTOTUNE)
                 .shuffle(int(10e3)))
eval_dataset = (dataset['test']
                .map(_preprocess)
                .batch(batch_size)
                .prefetch(tf.data.experimental.AUTOTUNE))

- So now we're back to a TF Dataset

In [ ]:
train_dataset

- And here you can see that the random binarization
  is different each time the data comes through as a batch

In [ ]:
# different each time since
# map is applied at run time

tmp=iter(train_dataset)
sm = 0
for i in tmp:
    sm += i[0].numpy().sum() 
sm

In [ ]:
# different each time since
# map is applied at run time

tmp=iter(train_dataset)
sm = 0
for i in tmp:
    sm += i[0].numpy().sum() 
sm

# Variational AutoEncoder (VAE)

$\tilde x = I(mvn(f_{\mu,\sigma}(x)))$

- $\tilde x$: reconstructed image
- $x$: input image
- $f_{\mu,\sigma}$: image embedder
- $mvn$: multivariate normal distribution
- $I$: image reconstructor





## Encoder

$\tilde x = I(mvn($<font style='color:red'>$f_{\mu,\sigma}(x)$</font>$))$

- $\tilde x$: reconstructed image
- $x$: input image
- <font style='color:red'> $f_{\mu,\sigma}$: image embedder</font>
- $mvn$: multivariate normal distribution
- $I$: image reconstructor



In [ ]:
print(train_images.shape)

print(list(train_images.shape[1:])+[1])

train_dataset


In [ ]:
# https://www.tensorflow.org/tutorials/generative/cvae
# https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE

from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, InputLayer, Flatten, Dense, Lambda

square_kernel_size = 5 #3
x_y_stride_offset = 2
base_filter_count = 32

encoder = Sequential([#Input(shape=list(train_images.shape[1:])+[1]),
                      InputLayer(input_shape=list(train_images.shape[1:])+[1],
                                 name='input'),
                      Lambda(lambda x: tf.cast(x, tf.float32) - 0.5),
    
                      Conv2D(filters=base_filter_count, kernel_size=square_kernel_size, 
                             strides=1, padding='valid', activation=tf.nn.leaky_relu),
    
                      Conv2D(filters=base_filter_count, kernel_size=square_kernel_size, 
                             strides=x_y_stride_offset, padding='same', activation=tf.nn.leaky_relu),
    
                      Conv2D(filters=2*base_filter_count, kernel_size=square_kernel_size, 
                             strides=1, padding='valid', activation=tf.nn.leaky_relu),
    
                      Conv2D(filters=2*base_filter_count, kernel_size=square_kernel_size, 
                             strides=x_y_stride_offset, padding='same', activation=tf.nn.leaky_relu),
    
                      Conv2D(filters=4*base_filter_count, kernel_size=4, 
                             strides=1, padding='valid', activation=tf.nn.leaky_relu),
                      Flatten()],
                     name='encoder')

print(encoder.input_shape)
encoder.summary()

### Parameter Count Check

In [ ]:
# 32 5x5[3x3] kernels, each with an offset, applied to a single image channel
channels = 1
filters = 32
params = 0
for i in range(filters):
    params += channels*square_kernel_size*square_kernel_size + 1
print(params)

# for each of 32 image channels, 64 5x5[3x3] kernels, with one offset for each image channel
channels = 32
filters = 64
params = 0
for i in range(filters):
    params += channels*square_kernel_size*square_kernel_size + 1
print(params)


### Strided Convolutions

In [ ]:
x_y_stride_offset=2
square_kernel_size=5

for i_x in range(12):
    for i_y in range(12):
        x_i_left = x_y_stride_offset*i_x
        x_i_right = x_i_left+square_kernel_size
        y_i_top = x_y_stride_offset*i_y
        y_i_bottom = y_i_top+square_kernel_size
        print("l: "+str(x_i_left)+", r: "+str(x_i_right)+", t: "+str(y_i_top)+", b:"+str(y_i_bottom))
        
# https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t        


## TensorFlow Probability

$\tilde x = I($ <font style='color:red'>$mvn(f_{\mu,\sigma}(x))$</font> $)$

- $\tilde x$: reconstructed image
- $x$: input image
- <font style='color:red'> $f_{\mu,\sigma}$: image embedder </font>
- <font style='color:red'> $mvn$: multivariate normal distribution </font>
- $I$: image reconstructor

- $mvn$: multivariate normal distribution

In [114]:
! pip install --upgrade tensorflow-probability

     |████████████████████████████████| 4.8 MB 3.1 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.11.1
    Uninstalling tensorflow-probability-0.11.1:
      Successfully uninstalled tensorflow-probability-0.11.1


In [ ]:
# https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE

# Super Key!
import tensorflow_probability as tfp

# https://blog.tensorflow.org/2019/03/regression-with-probabilistic-layers-in.html
# https://www.tensorflow.org/probability/api_docs/python/tfp/layers
tfpl = tfp.layers
tfd = tfp.distributions

# Important!
latent_dimension = 3


# https://www.tensorflow.org/probability/api_docs/python/tfp/layers/IndependentNormal
#tfd.Independent(tfd.Normal(loc=tf.zeros(latent_dimension), scale=1))
#tfd.Independent(tfd.Normal(loc=tf.zeros(latent_dimension), scale=1),
#                           reinterpreted_batch_ndims=1)
#prior = tfd.Independent(tfd.Normal(loc=tf.zeros(latent_dimension), scale=1),
#                        reinterpreted_batch_ndims=1)
# https://www.tensorflow.org/probability/api_docs/python/tfp/layers/MultivariateNormalTriL

# slight upgrade from i.i.d. normal prior specification (above) to MVN
prior = tfd.MultivariateNormalDiag(loc=tf.zeros(latent_dimension), scale_diag=tf.ones(latent_dimension))#tf.eye(latent_dimension))
# prior TO BE DISCUSSED SHORTLY
# along with notes such as this:
# https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/vae.py
# and as discussed from line 62 in the `vae.py` link above
# mu and sigma in f_mu,sigma can just change to accomodate different MNV(mu_0, sigma_0)
# so we just use MVN(0,1) for the "prior"


#mvn = Sequential([InputLayer(input_shape=encoder.output.shape[1:]),
#                  Dense(tfpl.IndependentNormal.params_size(latent_dimension), activation=None),
#                  tfpl.IndependentNormal(latent_dimension,
#                       activity_regularizer=tfpl.KLDivergenceRegularizer(prior))],
#                name='mvn')

# slight upgrade from i.i.d. normal prior specification (above) to MVN
mvn = Sequential([InputLayer(input_shape=encoder.output.shape[1:]),
                  Dense(tfpl.MultivariateNormalTriL.params_size(latent_dimension), activation=None),
                  tfpl.MultivariateNormalTriL(latent_dimension, 
                       activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=None,
                                                                         use_exact_kl=False))],
                name='mvn')
# as noted here: https://www.tensorflow.org/tutorials/generative/cvae#define_the_loss_function_and_the_optimizer
# "In practice, we optimize the single sample Monte Carlo estimate of this expectation"

print(mvn.input.shape)
mvn.summary()

## Parameter Counting

In [ ]:
# 2304 hidden nodes
# 1 bias offset coefficient term weight
# 9 paramters (3 means, 6 covariance parameters) for 3d independent multivariate normal distribution
(128+1)*9


# Distributional Components

Often discussed with *Bayesian* terminology, e.g., as here: `https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/vae.py`

- Image Distribution: $X \sim p_X$
    - the marginal data distribution
- Conditional Encoder Distribution: $Z|X \sim q_{Z|X}$
    - the "posterior"?
- Conditional Decoder Distribution: $\tilde X \sim p_{\tilde X|Z}$
    - the "likelihood"?
- Encoder Prior/Marginal Distribution: $Z \sim p_{Z}$
    - the "prior"?
    
<font style="color:blue">But this is not actually typical *Bayes* where we're given the **data** $x$, **likelihood** $f(x|z)$, and the **prior** $\pi(z)$ and use *Bayes Theorem* to derive the **posterior** $\pi(z|x)$... **i.e., $z$ characterizes everything we don't know**</font>


<font style="color:red">With VAEs, ***instead***, we learn 
the **decoder** "likelihood" $p_{\tilde X|Z}(x| Z)$,
the **encoder** "posterior" $q(Z|x)$,
and make the *latent (marginal)* **encoding distribution** $p(Z)$ "prior"
match the "posterior" $q(Z|x)$ (or, effectively/more accurately, *vice-versa*).</font>

<br>

<font style="color:green">
Note: that last part about "prior" matching the "posterior" feels more accurate if the prior is, e.g., a mixture model.
</font>


<br>

<font style="color:blue"> Anyway, in VAEs, $Z$ is just the embedding variable... but what we're actually learning is ***EVERYTHING*** for **EVERY possible** $X=x$ [not just $\pi(\textbf{z}|x)$]: </font>

<br>
<font style="color:blue">$$p_{\tilde X|Z}(x| Z), \;q(Z|x), \text{ and } p(Z)$$.</font>

*So this is more like nonparametric Bayes.*  I don't actually like the terminology because it suggests it's just about the latent embedding $Z$... 

*But it's [BOTH] much more than that...* and also less than that...

In [ ]:
# since we're specifying p(Z)
# we could experiment with specifying a mixture distribution for that (the "prior")
# Not pursuing this implementation right now

# https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MixtureSameFamily
# https://www.tensorflow.org/probability/api_docs/python/tfp/layers/MixtureNormal
# https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalTriL

if 0:
    number_components = 25
    latent_dimension = 3

    locs = tf.Variable(np.zeros([number_components, latent_dimension], dtype=np.float32), 
                       name="locs", trainable=True)
    mixture_logits = tf.Variable(np.zeros([number_components], dtype=np.float32),
                                 name="mixture_logits", trainable=True)

    # https://www.tensorflow.org/probability/examples/Learnable_Distributions_Zoo#mixture_of_multivariate_normal_full_cov
    #scale_trils = [tfp.util.TransformedVariable(tf.eye(3, dtype=tf.float32),
    #                                                     tfp.bijectors.FillScaleTriL(),
    #                                                     name='comp_'+str(i)+'_TriL') for i in range(number_components)] 
    #tf.stack(


    scale_trils=tfp.util.TransformedVariable(
                tf.eye(latent_dimension, batch_shape=[number_components]),
                bijector=tfp.bijectors.FillScaleTriL(),
                name='scale_trils')

    pr = tfd.MixtureSameFamily(
         components_distribution=
         tfd.MultivariateNormalTriL(loc=locs, scale_tril=scale_trils),
         mixture_distribution=tfd.Categorical(logits=mixture_logits),
         name="prior")



# Evidence Lower BOund (ELBO)

The standard way this is presented goes like this:
- `https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/vae.py`
- `https://en.wikipedia.org/wiki/Jensen%27s_inequality`
- `https://en.wikipedia.org/wiki/Kullback–Leibler_divergence`


$$
\begin{align*}
\log p(x) & = {} \log \int p(x|z) p(z) dz \\
& = {} \log \int \frac{p(x|z) p(z)}{\textbf{q(z|x)}} \textbf{q(z|x)} dz \\
& = {} \log \textrm{E}_{\textbf{Z}\sim \textbf{q(Z|x)}}\left[\frac{p(x|Z) p(Z)}{q(Z|x)}\right] \\
& \underset{so \; Jensen's}{\overset{\log concave}{\geq}} {}
 \textrm{E}_{Z\sim q(Z|x)}\left[\textbf{log} \frac{p(x|Z) p(Z)}{q(Z|x)}\right] 
 \quad \longleftarrow \quad \equiv {} 
 -\textrm{KL}[ \; q(Z|x) \; || \; p(x|Z)p(Z)\; ] \\
 & \;\;\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad= {} -\textrm{KL}[ \; q(Z|x) \; || \; p(x,Z)\; ] \\
& \underset{\sum \textrm{of the} \log \;}{\overset{\log \textrm{of the} \prod \;}{=}} {} \textrm{E}_{Z\sim q(Z|x)}\left[\log p(x|Z) + \log \frac{p(Z)}{q(Z|x)}\right] \\ 
& = {} \textrm{E}_{Z\sim q(Z|x)}\left[\log p(x|Z) \right] + \textrm{E}_{Z\sim q(Z|x)}\left[\log \frac{p(Z)}{q(Z|x)}\right] \\ 
& \equiv {} \textrm{E}_{Z\sim q(Z|x)}\left[\log p(x|Z) \right] - \textrm{KL}[ \; q(Z|x) \; || \; p(Z)\; ]\\
& = {} \textrm{E[Reconstruction log-likelihood]} - \textrm{KL}[ \; q(Z|x) \; || \; p(Z)\; ]\end{align*}
$$

- The bigger this is the better our model for $x$, $p(x)$ (since $p(x)$ $\geq$ `this`)
- since KL is $\geq 0$ we make $q(Z|x)$ as close to $p(Z)$ as possible
    - and maximize $p_{\tilde X|Z}(x| Z)$ (i.e., $\textrm{E[Reconstruction log-likelihood]}$)


# But it can also be derived this way...

- from which you can more clearly see that even $q(z|x)$ is not estimating $p(z|x)$ and hence further deviating from a Bayesian analysis...
- ...though we already knew that since $q(z|x)$ is trying to look like $p(z)$... which is not how a posterior distribution works... Anyway:

$$
\begin{align*}
\log p(x) & = {} \int \log p(x) q(z|x) dz \\
& = {} \int \log \frac{p(x|z) p(z)}{p(z|x)} q(z|x) dz \\
& = {} \int q(z|x) \left( \textbf{log} \frac{\textbf{p(x|z) p(z)}}{\textbf{p(z|x)}} \right) dz \\
& = {} \int q(z|x) \left( \log \frac{p(x|z) p(z)}{\textbf{q(z|x)}}\frac{\textbf{q(z|x)}}{p(z|x)} \right) dz \\
& = {} \int q(z|x) \left( \log \frac{p(x|z) p(z)}{q(z|x)} \textbf{ +} \log \frac{q(z|x)}{p(z|x)} \right) dz \\
& = {} \int q(z|x) \log \frac{p(x|z) p(z)}{q(z|x)} dz + \int q(z|x) \log \frac{q(z|x)}{p(z|x)} dz \\
& \equiv {} -\textrm{KL}[\; q(Z|x) \; || \; p(x|Z) p(Z) \;] - \textrm{KL}[\; q(Z|x) \; || \; p(Z|x) \;] \\
& \geq {} - \textrm{KL}[\; q(Z|x) \; || \; p(x|Z) p(Z) \;] \quad \text{$since$ KL $\geq 0$} \\
& = {} \int q(z|x) \log \frac{p(x|z) p(z)}{q(z|x)} dz \\
& = {} \int q(z|x) \log p(x|z) dz + \int q(z|x) \log \frac{p(z)}{q(z|x)} dz \\
& \equiv {} \textrm{E}_{Z\sim q(Z|x)}\left[\log p(x|Z) \right] - \textrm{KL}[\; q(Z|x) \; || \; p(Z) \;] \\ 
& = {} \textrm{E[Reconstruction log-likelihood]} - \textrm{KL}[ \; q(Z|x) \; || \; p(Z)\; ]
\end{align*}
$$

- and we've arrived at the same place as we did above

**So what we're doing is making a good model for $x$, $p(x)$.**
- We're not actually trying to learn posterior distributions for the latent variables $Z$, $p(Z|x)$;
- still, we did end up creating the "decoder" $p_{\tilde X|Z}(x| Z)$
- and as well creating $p(Z)$, which we can use to feed into that $p_{\tilde X|Z}(x| Z)$ "decoder"
    - and creating  a map $q(Z|x)$ into that space, which is how inform estimation of $p(Z)$ [if needed]


## Decoder

$\tilde x = $<font style='color:red'>$I$</font>$(mvn(f_{\mu,\sigma}(x)))$

- $\tilde x$: reconstructed image
- $x$: input image
- $f_{\mu,\sigma}$: image embedder
- $mvn$: multivariate normal distribution
- <font style='color:red'>$I$: image reconstructor</font>



In [ ]:
# https://www.tensorflow.org/tutorials/generative/cvae
# https://www.tensorflow.org/probability/examples/Probabilistic_Layers_VAE

from tensorflow.keras.layers import Reshape, Conv2DTranspose

decoder = Sequential([InputLayer(input_shape=(latent_dimension,)),
                      Dense(units=7*7*32, activation='relu'),
                      Reshape(target_shape=(7, 7, 32)),
                      Conv2DTranspose(filters=64, kernel_size=square_kernel_size, 
                                      strides=x_y_stride_offset, padding='same', activation='relu'),
                      Conv2DTranspose(filters=32, kernel_size=square_kernel_size, 
                                      strides=x_y_stride_offset, padding='same', activation='relu'),
                      Conv2DTranspose(filters=1, kernel_size=square_kernel_size, 
                                      strides=1, padding='same', activation=None)],
                     name='decoder')

decoder.summary()



# `Conv2DTranspose` versus `Conv2D`
`https://keras.io/api/layers/convolution_layers/convolution2d_transpose/`


- The need for transposed convolutions generally arises from 
    - the desire to use a transformation going in the opposite direction of a normal convolution, 
        - i.e., from ~~something that has **the shape of**~~ <u>*the output*</u> of some convolution 
        - to ~~something that has the **shape of**~~ <u>*its input*</u>  
    - **while maintaining a connectivity pattern that is compatible with the "original" convolution.**



# So, it maintains same connectivity patterns, just in the opposite direction
- `https://datascience.stackexchange.com/questions/6107/what-are-deconvolutional-layers`
- `https://github.com/vdumoulin/conv_arithmetic`


| |4x4 Image w/ 3x3 Convolutional Kernel|5x5 Padded Image w/ 3x3 2-stride Convolutional Kernel|
|-|:-:|:-:|
|Forward Convolution: Blue $\rightarrow$ Green | <img src=https://github.com/vdumoulin/conv_arithmetic/raw/master/gif/no_padding_no_strides.gif style="transform:rotate(180deg);"> | <img src=https://github.com/vdumoulin/conv_arithmetic/raw/master/gif/padding_strides.gif style="transform:rotate(180deg);"> |
|Standard Convolution|$4x4 \rightarrow 1x16 \cdot 16x4 = 1x4 \rightarrow 2x2$| $7x7 \rightarrow 1x49 \cdot 49x9 = 1x9 \rightarrow 3x3$ |
|Transposed Convolution|$4x4 \leftarrow 1x16 \cdot 16x4$ <font style="color:red">$\cdot 4x16$</font> $= [1x4 \leftarrow 2x2]$ <font style="color:red">$\cdot 4x16$</font> | $7x7 \leftarrow 1x49 \cdot 49x9$ <font style="color:red">$\cdot 9x49$</font> $= [1x9 \leftarrow 3x3]$ <font style="color:red">$\cdot 9x49$</font> |
|"Reverse" Convolution: Blue $\rightarrow$ Green| ![](https://i.stack.imgur.com/YyCu2.gif) | ![](https://i.stack.imgur.com/f2RiP.gif) | 

# `Conv2DTranspose`  doesn't actually do the convolutional sliding
- it just sets up the transpose multiplication specification, corresponding the forward convolution parameterization
- which is computationally much more efficient than operationalizing the implied convolution computations

In [ ]:
# https://stackoverflow.com/questions/61110188/how-to-display-a-gif-in-jupyter-notebook-using-google-colab
#from IPython.display import Image
#Image(open('VAE/f2RiP.gif','rb').read())

## Loss

$\tilde x = I(mvn(f_{\mu,\sigma}(x)))$

- <font style='color:red'>$\tilde x$: reconstructed image</font>
- <font style='color:red'>$x$: input image</font>
- $f_{\mu,\sigma}$: image embedder
- $mvn$: multivariate normal distribution
- $I$: image reconstructor

Each pixel ("randomly" on/off by our processing treatment above) will be predicted as a Bernoulli random variable:

$$ \huge Bernoulli(x, \tilde x) = \prod_{i,j} \left(\frac{1}{1+e^{-\tilde x_{ij}}}\right)^{x_{ij}}$$

$$
\begin{align*}
\tilde x = {} & \log \left(\frac{p}{1-p}\right)\\
e^{\tilde x} = {} &  \frac{p}{1-p} \\
e^{\tilde x}-pe^{\tilde x} = & {} p\\
e^{\tilde x} = {} & p + pe^{\tilde x}\\
e^{\tilde x} = {} & p \left(1+ e^{\tilde x}\right)\\
e^{\tilde x} = {} & p \\
\frac{e^{\tilde x}}{1+ e^{\tilde x}}  = {} & p \\
\frac{1}{1+ e^{-\tilde x}}  = {} & p
\end{align*}
$$


In [ ]:
likelihood = Sequential([InputLayer(input_shape=decoder.output.shape[1:]),
                         Flatten(), # need to have a flat layer leading into tfpl
# input is assumed to be logits, so `convert_to_tensor_fn` is just a reshape pass through
                         tfpl.IndependentBernoulli(event_shape=list(train_images.shape[1:])+[1], 
                                                   convert_to_tensor_fn=tfd.Bernoulli.logits)],
                       name='likelihood')
# https://www.tensorflow.org/probability/api_docs/python/tfp/layers/IndependentBernoulli
# https://github.com/tensorflow/tensorflow/issues/42589

likelihood.summary() 


## TensorFlow Probility Layers
- Output Distributions!
    - which can be instantiated as tensors
    - or, just used as distributions!

In [ ]:

negloglik = lambda x, rv_tilde_x: -rv_tilde_x.log_prob(x)



In [ ]:
# Can but seemlessly used for gradient calculations

if 0:
    opt = tf.optimizers.Adam(learning_rate=1e-3)

    with tf.GradientTape() as tape:
        loss = negloglik(var, x)
      gradients = tape.gradient(loss, model.trainable_variables)
      opt.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
from tensorflow.keras import Model 
vae = Model(inputs=encoder.input,
                outputs=likelihood(decoder(mvn(encoder(encoder.input)))))

vae.summary()


In [ ]:

vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=negloglik) # assumes (y,y-hat) will be passed

# train_dataset (x,y) is (x,x)
# x in (x,y) is used to create tilde-x=y-hat
# y in (x,y) is x
# passes (y,y-hat)==(x,tilde-x) to loss function
# tilde-x is a distribution
hist = vae.fit(train_dataset, epochs=3,
               validation_data=eval_dataset)


In [ ]:
current_image = next(iter(train_dataset))[0][0,...,0]
plt.imshow(~current_image, cmap=plt.cm.binary)


In [ ]:
import numpy as np

fig,ax = plt.subplots(2,2)
ax=ax.flatten()
for i in range(4):
    current_image_reconstruction = vae(current_image[np.newaxis,...,np.newaxis]).sample()
    ax[i].imshow(1-current_image_reconstruction[0,...,0], cmap=plt.cm.binary)


In [ ]:
current_image_reconstruction_ave = vae(current_image[np.newaxis,...]).mean()
plt.imshow(1-current_image_reconstruction_ave[0,...,0], cmap=plt.cm.binary)


In [ ]:
randomly_generated_image = likelihood(decoder(prior.sample(1))).mean()
plt.imshow(randomly_generated_image[0,...,0], cmap=plt.cm.binary)


In [ ]:
# https://plotly.com/python/3d-scatter-plots/

import plotly.express as px
import pandas as pd

fig = px.scatter_3d()

means = []
for i in range(10):
    embedding = mvn(encoder((train_images[train_labels==i,...,np.newaxis][:100,...]/255.).round(0))).mean().numpy()
    means += [np.mean(embedding, axis=0)]
    df = pd.DataFrame(embedding, columns=list('xyz'))
    df['c'] = i
    if i < 10:
        fig.add_scatter3d(x=df.x, y=df.y, z=df.z, mode='markers', name=str(i))
        
fig.show()


In [ ]:
fig,ax = plt.subplots(1,10, figsize=(30,30))
ax = ax.flatten()
for i in range(10):
    ax[i].imshow(likelihood(decoder(means[i][np.newaxis,...])).mode()[0,...,0])
